## Modular arithmetic essentials
### Benchmarks
Here is how you do modular arithmetic in python. Run the two cells below and note how different their execution times are. You should use `pow` instead of doing modular arithmetic with the elementary operators.

In [2]:
import time

s = time.time()
ans = 1002583 ** 939001 % 2003951
t = time.time()

print("calculating {} took {:.2f} seconds".format(ans, t - s))

calculating 1590868 took 3.85 seconds


In [3]:
s = time.time()
ans = pow(1002583,939001,2003951)
t = time.time()

print("calculating {} operation took {:.2f} seconds".format(ans, t - s))

calculating 1590868 operation took 0.00 seconds


### Encrypting a number

If we start with a generator `g` and raise it to a power `s` mod `n`, then in general we cannot determine `s` even if `g` and `n` are known. This is the discrete logarithm problem.

We can say we "encrypt" `s` using the scheme

$$ E(s) = g^{s} \pmod n $$

But this means we need to choose `g` and `n` carefully. If `g` is 3, `s` is 2, and `n` is a large number, we can intelligently guess what `s` is in this case. We want to pick `g` and `n` such that all members "wrap around." We won't concern ourselves with this here, but you should be aware of it. The other issue is not picking a sufficiently large `n`. 

### Computing the discrete logarithm
If we do `s_p = g^s mod n`, and make `g` and `n` public, then it is infeasible to compute `s`. But this is only true if `n` is sufficiently large.

In the following example, we provide `s_p`, `g`, and `n`. Use python to bruteforce the solution to find `s`.

$$g^{ac} = (g^{a})^{c} \pmod p$$

In [7]:
n = 9551
g = 5
encrypted_number = 5666

<font color='red'>**Assignment 1: Use python to bruteforce the solution to find `s`**</font>

In [9]:
# Your Solution and Answer Here

s = 0 

while (g ** s) % n != encrypted_number:
    s += 1

student_solution = s


In [10]:
assert pow(g, student_solution, n) == encrypted_number
print("student_solution is {}".format(student_solution))

student_solution is 2531


### Computing the discrete logarithm the smart way
The smart way to do optimal algorithms is to [copy and paste from stackoverflow](https://stackoverflow.com/a/58665206/2079806). You do not have to understand how the library works

<font color='red'>**Assignment 2: Break the cryptography below and obtain the secret number**</font>

In [12]:
n = 1000004119
g = 5
encrypted_number = 767805982

In [13]:
# Your Solution and Answer Here
from sympy import discrete_log

student_solution = discrete_log(n, encrypted_number, g) # log_g(encrypted_number) (mod n)


In [14]:
assert pow(g, student_solution, n) == encrypted_number
print("student_solution is {}".format(student_solution))

student_solution is 420


## How Zero Knowledge Addition Works

A Zero Knowledge Proof verifies a computation is true without revealing a the inputs to the computation.

That is, if a + b = c, then we want a "trapdoor function" E(x) such that E(a)■E(b) = E(c) where ■ is the binary operator of the group E(x) is an element of.

So to prove a + b = c without revealing a, b, or c, we apply A = E(a), B = E(b), C = E(c) and give (A, B, C) to a verifier.

The verifier will then check A ■ B == B. // is this a typo? shouldn't it be A ■ B == C ?

Although they don't know the values A, B, and C "came from", they do know the unencrypted values form a valid sum.

## Zero Knowledge Addition
The following property is very important. It lets us verify the addition of numbers (integers under addition) using another group: $g^x \pmod p$ under multiplication. Because of the discrete logarithm, we cannot easily go from $g^x \pmod p$ to recover x.

a + b = c is homomorphic to $g^ag^b = g^c \pmod p$.

$$ g^{a}g^{b} = g^{a + b} \pmod n $$

Try a few different values of `a` and `b` yourself to see it in action

In [36]:
# both a and b are prime
a = 3
b = 5
# example of case where it may be true sans final modulo
print(pow(g, a, n) * pow(g, b, n) == pow(g, a + b, n))  

# only one is prime
a = 2
b = 16
print(pow(g, a, n) * pow(g, b, n) == pow(g, a + b, n)) 

# neither a or b are prime
a = 4
b = 16
print(pow(g, a, n) * pow(g, b, n) == pow(g, a + b, n)) 

# both larger primes
a = 3779
b = 7727
print(pow(g, a, n) * pow(g, b, n) == pow(g, a + b, n)) 

False
False
False
False


Wait what?! That's supposed to be true?

<font color='red'>**Assignment 3: The code above has a bug. What is the bug?**</font>

In [31]:
# Your corrected code here
# confirm correct 
print(pow(g, a, n) * pow(g, b, n) % n == pow(g, a + b, n)) 

True


## Zero Knowledge Subtraction
We can also encrypt the operation a - b. This is the same as 

$$ g^{a}g^{-b} = g^{a-b} $$

But we can't just stick a negative sign in front of the exponent

In [33]:
g = 57
n = 101
g ** -5 % n

1.6619797259514097e-09

The above is not an integer! However, python is smart enough to know what you are doing if you use `pow`. To test this, we expect

$$ g^{-a}g^{a} = 1 $$

because g^0 is 1, and `a - a = 0`.

In [139]:
a = 22
pow(g, a, n) * pow(g, -a, n) % n == 1

True

So what magic is happening behind the scenes? The formula below is used to compute the modular inverse.

$$ a^{-1} = a^{n - 2} \pmod n $$

In [148]:
a_inv = a ** (n - 2) % n
a_inv * a % n == 1

True

## Multiplication by a constant
Multiplication by a constant is really just repeated addition

$$ (g^{a})^{4} = g^{a} g^{a} g^{a} g^{a} $$

This allows us to "multiply by a constant" in a zero knowledge fashion.

In [153]:
a = 15
pow(g, a, n) * pow(g, a, n) * pow(g, a, n) * pow(g, a, n) % n == pow(g, a * 4, n)

True

Of course, it would be annoying to multiply like that if you have a big coefficient, so the following is better

In [155]:
pow(pow(g, a, n), 4 , n) == pow(g, a * 4, n)

True

## I know the solution to a systems of equations

Now you should be able to prove to me that you know the solution too in a zero knowledge fashion # corrected 'to' to 'too'

$$
2x + 8y = 7944\\
5x + 3y = 4764
$$

<font color='red'>**Assignment 4: convert the above equation to a zk proof system where you can prove to a verifier (with an agreed upon g and n) that you know the solution to an agreed upon system of equations, without revealing the solution**</font> # added 't' to 'conver' and s to 'system of equation'

In [3]:
g = 57
n = 101
r_1 = 7944
r_2 = 4764

# 2x + 8y = 7944
# 5x + 3y = 9528

# 5(2x + 8y) = 5(7944)
# 2(5x + 3y) = 2(4764)

# 10x + 40y = 39720
# 10x + 6y = 9528

#(10x + 40y) - (10x + 6y) = 39720 - 9528

# 34y = 30192
# 2x = 840

y = 888
x = 420

# (g^x)^2 == g^x * g^x == pow(g, x, n) * pow(g, x, n) == pow(g, 2 * x, n) 
# pow(g, 8 * y, n), etc

# confirm below are correct
assert(pow(g, 2 * x, n) * pow(g, 8 * y, n) % n  == pow(g, r_1, n)) 
assert(pow(g, 2 * x + 8 * y, n) % n == pow(g, r_1, n)) 

assert(pow(g, 5 * x, n) * pow(g, 3 * y, n) % n == pow(g, r_2, n)) 
assert(pow(g, 5 * x + 3 * y, n) % n == pow(g, r_2, n)) 

print(pow(g, r_1, n) % n) # 87
print(pow(g, r_2, n) % n) # 87

# demonstrating with more equations to illustrate different values
r_3 = 7 * x + 11 * y # 12708

assert(pow(g, 7 * x + 11 * y, n) % n == pow(g, r_3, n)) 

print(pow(g, r_3, n) % n) # 95

r_4 = 3 * x + 37 * y  # 34116

assert(pow(g, 3 * x + 37 * y, n) % n == pow(g, r_4, n)) 

print(pow(g, r_4, n) % n) # 36


# so would the 'proof' be providing the the value of 'r' and the value of pow(g, r, n) % n ?

87
87
95
36


## Convert a rational number to a finite field element

<font color='red'>**Assignment 5: Compute 53/192 + 61/511 (mod 1033) in python. Show your work. You are not required to implement euclid's algorithm. Show it is equal to the original rational number sum: https://www.wolframalpha.com/input?i=53%2F192+%2B+61%2F511**</font>

In [6]:
p = 1033 

n_1 = 53
n_2 = 61

d_1 = 192
d_2 = 511

# calculate inverse of denomitor
d_1i = pow(d_1, -1, p)
d_2i = pow(d_2, -1, p)

# confirm inverse is calculated correctly
assert(d_1 * d_1i % p == 1) 

# for each fraction, multiply the numerator by the inverse of the denominator 
f_1 = n_1 * d_1i
f_2 = n_2 * d_2i 

# take the sum modulo p
f_s = (f_1 + f_2) % p

print(f_s) # is this correct? if so, how to show it is equal to the original number sum?

514
